In [1]:
import pdfplumber
import re
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
def extract_table_and_emails(pdf_path):
    data = []
    email_links = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            
            if text:
                lines = text.split("\n")
                for line in lines:
                    match = re.match(r"(\d+)\s+([A-ZÁÉÍÓÚÑ,.'\s-]+)\s+(\d{9})\s+(\*\*Inscrito por Web\*\*)\s+(Licenciatura|Maestría|Doctorado)\s+(\d+\.\d+)(.*)?", line)
                    
                    if match:
                        record = list(match.groups())
                        if len(record) == 6:
                            record.append("No disponible")
                        data.append(record)
            
            # Extraer correos electrónicos
            if page.annots:
                for annot in page.annots:
                    uri = annot.get("uri", "")
                    if uri.startswith("mailto:"):
                        email_links.append(uri.replace("mailto:", ""))
    
    if data:
        df = pd.DataFrame(data, columns=["Número de Registro", "Nombre de Alumno", "ID", "Status de Inscripción",
        "Nivel", "Créditos", "Detalle de Calificaciones"])
    else:
        print("No se encontraron datos en la tabla. Verifica la estructura del PDF.")
        return None
    
    df.dropna(how="all", inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    unique_emails = list(dict.fromkeys(email_links))
    
    if len(unique_emails) >= len(df):
        df["Correo Electrónico"] = unique_emails[: len(df)]
    else:
        df["Correo Electrónico"] = ["No disponible"] * len(df)
        for i in range(min(len(df), len(unique_emails))):
            df.at[i, "Correo Electrónico"] = unique_emails[i]
    
    # Renombrar la columna "Correo Electrónico" a "Correo"
    df.rename(columns={"Correo Electrónico": "Correo"}, inplace=True)

    
    return df

In [3]:
def save_to_excel(df, output_file):
    if df is not None:
        wb = Workbook()
        ws = wb.active
        ws.title = "Lista de Estudiantes"
        
        # Escribir encabezados una sola vez
        ws.append(df.columns.tolist())
        
        # Escribir datos fila por fila
        for row in dataframe_to_rows(df, index=False, header=False):
            ws.append(row)
        
        # Guardar el archivo Excel
        wb.save(output_file)
        print(f"Archivo Excel generado correctamente: {output_file}")
    else:
        print("No se generó el archivo Excel debido a errores en la extracción de datos.")


In [4]:
# Ruta del archivo PDF
pdf_path = "Lista_Trabajo_Colaborativo.pdf"
output_excel = "resultado_lista.xlsx"


In [5]:
# Ejecutar la extracción
df_result = extract_table_and_emails(pdf_path)


In [6]:
# Guardar el resultado en un archivo Excel con formato de tabla
save_to_excel(df_result, output_excel)


Archivo Excel generado correctamente: resultado_lista.xlsx
